In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.2'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [773 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,085 kB]
Get:13 http://archive.ubuntu.com/ub

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)



In [6]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")


In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query1 = """
SELECT Year(Date) AS Year, ROUND(AVG(Price), 2) AS Average_Price
FROM home_sales
WHERE Bedrooms = 4
GROUP BY Year
"""
spark.sql(query1).show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [9]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query4 = """
SELECT year(to_date(Date_Built, 'yyyy-MM-dd')) AS Year, ROUND(AVG(Price), 2) AS Average_Price
FROM home_sales
WHERE Bedrooms = 3 AND Bathrooms = 3
GROUP BY Year
"""
spark.sql(query4).show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|null|    292003.61|
+----+-------------+



In [11]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query5 = """
SELECT year(to_date(Date_Built, 'yyyy-MM-dd')) AS Year, ROUND(AVG(Price), 2) AS Average_Price
FROM home_sales
WHERE Bedrooms = 3 AND Bathrooms = 3 AND Floors = 2 AND sqft_living >= 2000
GROUP BY Year
"""
spark.sql(query5).show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|null|    292538.42|
+----+-------------+



In [12]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query4 = """
SELECT View, ROUND(AVG(Price), 2) AS Average_Price
FROM home_sales
GROUP BY View
HAVING AVG(Price) >= 350000
ORDER BY View DESC
"""
spark.sql(query4).show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|View|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 1.7532875537872314 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
spark.sql(query4).show()
print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|View|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.5668888092041016 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("Date_Built").parquet("/content/home_sales_partitioned")


In [17]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("/content/home_sales_partitioned")

In [18]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
start_time = time.time()

spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS AveragePrice
    FROM home_sales_parquet
    GROUP BY view
    HAVING AveragePrice >= 350000
    ORDER BY view DESC
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|AveragePrice|
+----+------------+
| 100|   1026669.5|
|  99|  1061201.42|
|  98|  1053739.33|
|  97|  1129040.15|
|  96|  1017815.92|
|  95|   1054325.6|
|  94|   1033536.2|
|  93|  1026006.06|
|  92|   970402.55|
|  91|  1137372.73|
|  90|  1062654.16|
|  89|  1107839.15|
|  88|  1031719.35|
|  87|   1072285.2|
|  86|  1070444.25|
|  85|  1056336.74|
|  84|  1117233.13|
|  83|  1033965.93|
|  82|   1063498.0|
|  81|  1053472.79|
+----+------------+
only showing top 20 rows

--- 0.9170963764190674 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [22]:
# 15. Check if the home_sales is no longer cached
print("Is home_sales cached? ", spark.catalog.isCached("home_sales"))


Is home_sales cached?  False
